In [16]:
from nglui.statebuilder import *
import pandas as pd
from matplotlib import cm
import matplotlib
import numpy as np

In [17]:
img_source = 'precomputed://gs://zetta_lee_fly_vnc_001_precomputed/vnc1_full_v3align_2/realigned_v1'
img_layer = ImageLayerConfig(name='img',
                             source=img_source,
                             )

#### Read in MN Table and get MN_A101_T1R

In [18]:
soma_df = pd.read_csv('/Users/brandon/Documents/Repositories/Python/Neuron_Database/T1MN_somas.csv')
soma_df

,Unnamed: 0,name,cell_type,pt_position,pt_root_id,soma_x_nm,soma_y_nm,soma_z_nm,found
0,0,MN_A101_T1L,NaN,"[13329, 114635, 1861]",648518346704067890,57314.7,492930.5,83745.0,NaN
1,1,MN_A101_T1R,NaN,"[63394, 114393, 2304]",648518346662254153,272594.2,491889.9,103680.0,NaN
2,2,MN_A102_T1L,NaN,"[11729, 116735, 1478]",648518346682057412,50434.7,501960.5,66510.0,NaN
3,3,MN_A102_T1R,NaN,"[63084, 116459, 2863]",648518346676423055,271261.2,500773.7,128835.0,NaN
4,4,MN_A103_T1R,NaN,"[59659, 115855, 2065]",648518346702534824,256533.7,498176.5,92925.0,NaN
...,...,...,...,...,...,...,...,...,...
116,116,MN_V402_T1L,NaN,"[23167, 87333, 2184]",648518346680460455,99618.1,375531.9,98280.0,NaN
117,117,MN_V501_T1L,NaN,"[27494, 86790, 2515]",648518346661954522,118224.2,373197.0,113175.0,NaN
118,118,MN_V501_T1R,NaN,"[54883, 84397, 2319]",648518346666755657,235996.9,362907.1,104355.0,NaN
119,119,MN_V601_T1R,NaN,"[49727, 85190, 2264]",648518346662417225,213826.1,366317.0,101880.0,NaN


In [19]:
# The name of each MN bundle is determined by the first 2 characters after MN_ so, if we want to see how many there are, 
# let's look at the set of options using set.
bundles = set([i[3:5] for i in soma_df.name])
print(bundles)


{'V4', 'A1', 'V1', 'L4', 'Ll', 'A2', 'D2', 'L3', 'V6', 'A5', 'D1', 'V2', 'A4', 'L1', 'V5', 'A3', 'L2', 'L5'}


#### Let's make a dataframe for each of these bundles.
##### We also want to specify a unique color to each. Neuroglancer uses hex color codes, so we need to convert. 

In [20]:
# Set up a colormap. cm is the colormap module. We want a color for each bundle. Use np.linspace to make an evenly spaced selection
# with the number of colors equal to the length of bundles. 
cmap = cm.hsv(np.linspace(0,1,len(bundles)))
dfs = []
idx = 0
for i in bundles:
    df = soma_df[soma_df['name'].str.contains(i)]
    df['color'] = cm.colors.rgb2hex(cmap[idx])
    dfs.append(df)
    idx+=1
    

/Users/brandon/.local/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


#### Generate layers for each bundle

In [21]:
sbs = []

for i in bundles:
    seg_source = 'graphene://https://dev.zetta.ai/wclee/segmentation/table/vnc1_full_v3align_2'
    seg_layer = SegmentationLayerConfig(name = 'Bundle' + i,
                                        source = seg_source,
                                        selected_ids_column='pt_root_id',
                                        color_column='color')

    sbs.append(StateBuilder(layers=[img_layer, seg_layer]))


In [22]:
chained_sb = ChainedStateBuilder(sbs)
chained_sb.render_state(dfs, return_as='html')

In [27]:
sb = StateBuilder(layers=[img_layer, seg_layer],resolution=[4.3,4.3,45])
sb.render_state(soma_df.iloc[0:2], return_as='html')

In [14]:
anno_layer = AnnotationLayerConfig(name='annos',
                                   mapping_rules=points )


# Make a basic segmentation source
seg_layer = SegmentationLayerConfig(seg_source)

sb = StateBuilder([img_layer, seg_layer, anno_layer])
sb.render_state(soma_df, return_as='html')

,id,pre_root_id,post_root_id,cleft_vx,ctr_pt_x_nm,ctr_pt_y_nm,ctr_pt_z_nm,pre_pos_x_vx,pre_pos_y_vx,pre_pos_z_vx,ctr_pos_x_vx,ctr_pos_y_vx,ctr_pos_z_vx,post_pos_x_vx,post_pos_y_vx,post_pos_z_vx
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50478.604651,112639.302326,2206.0
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57217.906977,112803.488372,2518.0
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,50504.186047,112824.883721,2184.0
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,57431.860465,112768.139535,2551.0
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,58558.372093,111946.279070,2593.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
934,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,51697.441860,105024.883721,1987.0
935,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53542.558140,105817.674419,1963.0
936,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53776.279070,105624.651163,1970.0
937,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,53699.302326,105622.558140,1970.0


In [32]:
#First state builder
seg_layer = SegmentationLayerConfig(name = 'MNA101_T1R',
                                    source = seg_source,
                                    selected_ids_column='pt_root_id')

neuron_sb = StateBuilder(layers=[img_layer, seg_layer],resolution=[4.3,4.3,45])

points = PointMapper(point_column='xyz')
postsyn_annos = AnnotationLayerConfig('post', color='#00CCCC', mapping_rules=points)

postsyn_sb = StateBuilder(layers=[img_layer, postsyn_annos])


# Chained state builder
chained_sb = ChainedStateBuilder([neuron_sb, postsyn_sb])
chained_sb.render_state([soma_df.iloc[0:2], synapse_table], return_as='html')

,Unnamed: 0,name,cell_type,pt_position,pt_root_id,soma_x_nm,soma_y_nm,soma_z_nm,found
0,0,MN_A101_T1L,NaN,"[13329, 114635, 1861]",648518346704067890,57314.7,492930.5,83745.0,NaN
1,1,MN_A101_T1R,NaN,"[63394, 114393, 2304]",648518346662254153,272594.2,491889.9,103680.0,NaN
2,2,MN_A102_T1L,NaN,"[11729, 116735, 1478]",648518346682057412,50434.7,501960.5,66510.0,NaN
3,3,MN_A102_T1R,NaN,"[63084, 116459, 2863]",648518346676423055,271261.2,500773.7,128835.0,NaN
4,4,MN_A103_T1R,NaN,"[59659, 115855, 2065]",648518346702534824,256533.7,498176.5,92925.0,NaN
...,...,...,...,...,...,...,...,...,...
116,116,MN_V402_T1L,NaN,"[23167, 87333, 2184]",648518346680460455,99618.1,375531.9,98280.0,NaN
117,117,MN_V501_T1L,NaN,"[27494, 86790, 2515]",648518346661954522,118224.2,373197.0,113175.0,NaN
118,118,MN_V501_T1R,NaN,"[54883, 84397, 2319]",648518346666755657,235996.9,362907.1,104355.0,NaN
119,119,MN_V601_T1R,NaN,"[49727, 85190, 2264]",648518346662417225,213826.1,366317.0,101880.0,NaN
